# Data Analysis with Python, Pandas, and Jupyter Notebook

JupyterLab is a convenient interface for interactive coding.

It allows you to mix text (including $\LaTeX$ for math, if you're into that), code, results of the code, graphics, and even interactive widgets.

![PyData Stack](http://chris35wills.github.io/courses/pydata_stack.png)

## 1. Using the Notebook [0:01]

One difference from regular Python is that whereas regular Python (inculding on Codecademy) will not display any output until you type `print`, Jupyter notebook will output the last _expression_ of each cell.

Most code that calculates a value is an expression. Assignments (=) are not expressions.

For example,

In [ ]:
2+2

In [ ]:
1

In [ ]:
"hello" + " " + "world" + " " + str(42)

In [ ]:
x = 21

You can use the toolbar to add and edit cells. You can run a cell by pressing Ctrl-Enter.

The drop-down selector for "Markdown" means text. "Code" is code.

## 2. Looking at our Data [0:02]

The library we use is called Pandas, short for "Python data analysis."

We usually shorten it to `pd`, so we can call its method using `pd.` instead of `pandas.`.

You can read its documentation here: http://pandas.pydata.org/pandas-docs/stable/. <-- Keep this website open as you play with Pandas. It is the official documentation for the Pandas library and is comprehensive. A Google search will help you find examples for how to use code, but the official documentation will give you the straight dope on how everything works, which is important because Pandas is a huge library with many details.

We will use _real_, de-identified physiological data from ICU patients from https://physionet.org/challenge/2012/. <-- Open this to see the data dictionary. Always get a data dictionary and avoid guessing what numbers mean when you can (but many times you can't...)

Pandas can read files directly from URLs, so we don't need to download and upload this file to our laptops.

In [ ]:
#  This lets us use the library pandas shortened as pd
import pandas as pd
#  This creates a data frame called 'df' with values loaded from online
df = pd.read_csv("https://physionet.org/challenge/2012/set-a/132551.txt")


# Pandas uses Matplotlib for plotting. This line below is special IPython Notebook syntax (magic) to show plots in the notebook.
%matplotlib inline

Jupyter notebook automatically prints the DataFrame as an HTML table. Compare this to viewing the file in your browser.

In [ ]:
df

If your tables are much bigger than this one, printing it all out may be slow. You can instead do

In [ ]:
# Shows first 5 entries
df.head(5)

In [ ]:
# Shows last 5 entries
df.tail(5)

A DataFrame consists of multiple columns and rows. We can see the row labels with the index attribute.  We can see which columns the data frame currently has using the columns attribute (notice no parentheses)

In [ ]:
df.index #row labels

In [ ]:
df.columns #column labels

We see that the data frame has three columns labeled 'Time', 'Parameter', and 'Value'.  While the rows are labeled 0 to 672.  

We can pull specific values from the data frame by selecting the row and column accordingly using the loc (location) and iloc (indexed location) meathods:

In [ ]:
df.loc[4,'Parameter']

In [ ]:
df.iloc[4,1]

We can select individual rows by only entering the row into loc or iloc:

In [ ]:
df.loc[4]

Similarly, you can call a column using:

In [ ]:
df.loc[:,'Parameter']

Notice that ':' in the row stands for 'everything'.  A useful shortcut is to just call into the data frame like so:

In [ ]:
df['Parameter']

In [ ]:
df['Value']

A column is a a new type: Series. In fact, all Series have the same type

In [ ]:
type(df['Parameter'])

In [ ]:
type(df['Value'])

But different Series can contain different types of data. The type of the data contained inside the Series is the `.dtype`.

In [ ]:
df['Parameter'].dtype

``'O'`` means "Python object", but in general, it usually means string.

In [ ]:
df['Value'].dtype


**EXERCISE**: Select the Time column.

What is the type of the Time column?

What type of data does the Time column contain?

In [ ]:
#place code here!

### 2.1 Cleaning up Datetimes [0:10]

**What did we notice about the Time column?**

Cleaning up datetimes is a common bane of data analysis. Fortunately, Pandas was developed by a trader, so it has excellent support for time series data. Physiological data are also time series data.

How do we interpret the numbers in Time? Don't guess! Read the data dictionary:

> Each observation has an associated time-stamp indicating the elapsed time of the observation since ICU admission in each case, in hours and minutes. Thus, for example, a time stamp of 35:19 means that the associated observation was made 35 hours and 19 minutes after the patient was admitted to the ICU.

So Time is not an absolute datetime, but rather a _time delta_ relative to the start of the patient's ICU stay.

Pandas has a special data type to represent this phenomenon: `Timedelta`. (`DateTime` is for absolute datetimes, i.e. '2018-11-29 11:00:00').  You can easily convert a column to a timedelta type using pd.to_timedelta()

In [ ]:
pd.to_timedelta(df['Time'])

Hmmm....

In [ ]:
df['Time'].head()

Let's just try adding :00 to the end of everything:

In [ ]:
pd.to_timedelta(df['Time'] + ":00")

It worked!

However if we look at the data frame the values remain unchanged...

In [ ]:
#note the dataframe is unchanged!
df.head()

In order for the data frame we have to reassign the values of the column.

Before we do that though let's save our results to a new DataFrame.

In general, I try to avoid overwriting DataFrames. Instead, I create new ones as as I go. This ensures that I can always backtrack to an earlier stage of the data manipulation if something didn't work or to explore something else, without having to restart the program from scratch.

Name variables descriptively, **NOT** df2, df3, ...

In [ ]:
df_td = df.copy()

#we assign the column to the data frame
df_td['Time'] = pd.to_timedelta(df_td['Time'] + ":00")

When working with time series data in Pandas, we must set the _index_ to be the time to enable Pandas to be smart about datetime.

One thing Pandas can do intelligently with a DateTime or Timedelta index is plotting, which we will explore later.

In [ ]:
df_td_ix = df_td.set_index('Time')
df_td_ix

In [ ]:
df_td_ix.index

Notice that the rows are labeled differently and the data frame has one less column(which will change iloc).  As well as loc has to now take one of the earlier selected values now:

In [ ]:
df_td_ix.iloc[4,0] # vs df.iloc[4,1]

In [ ]:
df_td_ix.loc['00:05:00']

### 2.3 Queries [0:20]

**What are the different kinds of Parameters?**

Just like you can call methods on strings, you can call methods on columns (Series)

In [ ]:
df_td_ix['Parameter'].unique()

Recall that ``df['Parameter']`` is a Series:

In [ ]:
df_td_ix['Parameter']

Note that since we set `df_td_ix` to have a Timedelta index, every column we extract from `df_td_ix` will now have the SAME Timedelta index.

Compare to

In [ ]:
df['Parameter']

which only shows the row number, which is less useful than directly showing the Timedelta.

**How many measurements of each do we have?**

You can write SQL-like code in Pandas. It will not be as elegant as SQL (if you think SQL is elegant, that is), but in addition to what you can do in SQL, you can do anything else you want in Python.

        select Parameter, count(*) from df group by Parameter order by count(*) desc
        
        
If you are interested in more advanced operations, take a look here: https://medium.com/jbennetcodes/how-to-rewrite-your-sql-queries-in-pandas-and-more-149d341fc53e


In [ ]:
# This counts all the occurences of each parameter type in the Parameter column and sorts it. 
df_td_ix.groupby('Parameter')['Parameter'].count().sort_values(ascending=False)

For example, we can save this to a new variable.

Remember that whie in regular Python no output is shown unless you use ``print``, Jupyter Notebook will by default print the last line in each cell.

In [ ]:
counts = df_td_ix.groupby('Parameter')['Parameter'].count().sort_values(ascending=False) # this doesn't print anything
counts # but this does

# If we uncomment the line below, then we no longer see counts, but we will see 2.
# 1 + 1 

### 2.3 Boolean Series and Selections [0:25]

We can do logical expressions on columns.

Notice how typing ``counts > 10`` evaluated _every_ entry of ``counts``. This is known as a _vectorized operation_. This is because ``counts`` behaves like a mathematical vector.

In [ ]:
frequent_params = counts > 10
frequent_params

If a Series contains only bool elements, we can use it as an index using the ``.loc`` syntax: (loc = locate)

In [ ]:
counts.loc[frequent_params]

We can select the first and last elements from a Series.

In [ ]:
hr_rows = df_td_ix['Parameter'] == 'HR'
hr_rows

The length of `hr_rows` equals the number of rows of `df`:

In [ ]:
len(hr_rows)

In [ ]:
df_td_ix.shape

DataFrames have two dimensions. We can select rows or columns or both.

Remember `:` means "select everything in this dimension"

In [ ]:
df_td_ix.loc[:, 'Value'] # Equivalent to df_td_ix['Value']

In [ ]:
hr_df_td_ix = df_td_ix.loc[hr_rows, :]
hr_df_td_ix

In [ ]:
#alternatively you can do it all in one go
df_td_ix.loc[df_td_ix['Parameter'] == 'HR',:]

In [ ]:
hr_df_td_ix.shape

In [ ]:
hr_rows.sum()

We can treat `bool` Series as Series of 1s and 0s.

**EXERCISE**: Explain the above two statements

**EXERCISE**: Pick a different parameter, and select all the rows of `df` corresponding to that parameter.

## 3. Plotting and Pivoting [0:35]

Recall that. Pandas actually uses Matplotlib under the hood for plotting. If you ran `%matplotlib inlne`, you can just use the `.plot()` methods, which do some of the work for you.


In [ ]:
#we can see the heart rate over time
hr_df_td_ix['Value'].plot()

In [ ]:
#we can see the frequency of different heart rate ranges over the different measurements
hr_df_td_ix['Value'].hist()

The long format is not very convenient. Let's pivot it to wide format. This is like a PivotTable in Excel:

In [ ]:
df_wide = df_td_ix.pivot(columns='Parameter', values='Value')
df_wide

RecordID is a unique number that identifies each visit. Since we only loaded the data for one visit, let's get rid of this.

In [ ]:
df_wide['RecordID'].unique()

In [ ]:
del df_wide['RecordID']

Pivoting revealed the gaps in our data: each cell represents a measurement of some value at some time. If a parameter, e.g. 'Age' is not measured at '1 days 14:18:00', then it shows up a `NaN` (like `NA` in R).

Usually for physiological data, we forward-fill:

In [ ]:
df_wide_filled = df_wide.ffill()
df_wide_filled

Now we can plot _everything_!

In [ ]:
df_wide_filled.plot()

Hmm, that wasn't very useful because the different data are on different scales.

One useful Pandas features is to compute summary statistics of everything at once. This method by default will ignore NaNs, which is what we want, since the NaNs are holes in the data.

**CAUTION**: To compute correct summary statistics, we must use `df_wide`, the DataFrame with "holes", NOT `df_wide_filled`. We don't want the filled-in data to count multiple times. 

In [ ]:
summary = df_wide.describe()
summary

By default, Pandas will hide extra columns. If we want to see EVERYTHING, we need to set an option:

In [ ]:
pd.options.display.max_columns = 999
summary

The arterial blood pressures are on a similar scale, so let's just plot those:

Clinical reference:
- MAP = mean arterial pressure
- DiasABP = diastolic arterial blood pressure
- SysABP = systolic arterial blood pressure
- NI = non-invasive

In [ ]:
summary[['MAP', 'DiasABP', 'SysABP', 'NIMAP', 'NIDiasABP', 'NISysABP']]

In [ ]:
df_wide_filled[['MAP', 'DiasABP', 'SysABP', 'NIMAP', 'NIDiasABP', 'NISysABP']]

In [ ]:
df_wide_filled[['NIDiasABP', 'NIMAP', 'NISysABP', 'DiasABP', 'MAP', 'SysABP']].plot()

As expected, the mean pressures fall between the diastolic and systolic. Also, the noninvasive pressures are usually lower than the corresponding invasive pressures. (Noninvasive is measured with a cuff; invasive is measured with a balloon inserted into the artery.)

In [ ]:
df_wide_filled[['NIDiasABP', 'NIMAP', 'NISysABP']].plot()

In [ ]:
df_wide_filled[['DiasABP', 'MAP', 'SysABP']].plot()

Sometimes the noninvasive measurements differs quite a bit from the invasive ones... I wonder which one is more accurate?

Another fun plot is a scatter matrix: a grid of scatterplots for every possible pair of columns. Let's try one with the blood pressures, and throw in heartrate for good measure:

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(df_wide_filled[['HR', 'NIDiasABP', 'NIMAP', 'NISysABP', 'DiasABP', 'MAP', 'SysABP']], figsize=(12,12))

We see that heartrate has some correlation with each blood pressure measurement. Also, the blood pressures are highly correlated for the same level of invasiveness, but not across the two levels of invasiveness. 

This is a problem. Noninvasive blood pressures should be highly correlated with invasive blood pressures. Assuming that our invasive measurements are correct (which is most likely the case), this may mean that our noninvasive measurements (cuffs) are inaccurate.

## 4. Transforming Data, Writing Output [0:50]

Recall our summary:

In [ ]:
summary

Let's pull out just the mean and standard deviations for each parameter:

In [ ]:
summary.loc['mean', :]

In [ ]:
summary.loc['std', :]

In [ ]:
type(summary.loc['mean', :])

Recall that if we select a row from a DataFrame, we get a Series indexed by the columns of the DataFrame.

Recall again that if we select a column from a DataFrame, we get a Series indexed by the rows of the DataFrame:

In [ ]:
summary.loc[:, 'MAP']

Because `summary.loc['mean', :]` and `summary.loc['std', :]` are indexed by the columns of `df_wide_filled`, Pandas is smart enough to figure out what we mean when we type

In [ ]:
df_wide_filled - summary.loc['mean', :]

Let's verify the computation for one cell:

In [ ]:
df_wide_filled.loc['0 days 00:50:00', 'BUN']

In [ ]:
summary.loc['mean', 'BUN']

In [ ]:
df_wide_filled.loc['0 days 00:50:00', 'BUN'] - summary.loc['mean', 'BUN']

Continuing, we can do

In [ ]:
df_wide_filled_zscores = (df_wide_filled - summary.loc['mean', :]) / summary.loc['std', :]
df_wide_filled_zscores

Note that this generated some extra NaNs (holes). Why?

In [ ]:
summary.loc[['count', 'std'], ['ALP', 'ALT', 'AST', 'Age', 'Albumin']]

We only ever have one measurement of these variables. The standard deviation of a sample of size 1 is undefined.

Finally, let's save our work:

In [ ]:
df_wide_filled_zscores.to_csv('filled_zscores.csv')